In [4]:
import pandas as pd
from haversine import haversine

north_pole = (90,0)
weight_limit = 1000.0

def bb_sort(ll): 
    ll = [[0,north_pole,10]] + ll[:] + [[0,north_pole,10]] 
    for i in range(1,len(ll) - 2):
        lcopy = ll[:]
        lcopy[i], lcopy[i+1] = ll[i+1][:], ll[i][:]
        if path_opt_test(ll[1:-1]) > path_opt_test(lcopy[1:-1]):
            ll = lcopy[:]
    return ll[1:-1]

def path_opt_test(llo):
    f_ = 0.0
    d_ = 0.0
    l_ = north_pole
    for i in range(len(llo)):
        d_ += haversine(l_, llo[i][1])
        f_ += d_ * llo[i][2]
        l_ = llo[i][1]
    d_ += haversine(l_, north_pole)
    f_ += d_ * 10 #sleigh weight for whole trip
    return f_

gifts = pd.read_csv("../data/gifts.csv").fillna(" ")
gifts['TripId']=0
gifts['i']=0
gifts['j']=0


for n in [1.2626]:
    i_ = 0
    j_ = 0
    for i in range(90,-90,int(-180/n)):
        i_ += 1
        j_ = 0
        for j in range(180,-180,int(-360/n)):
            j_ += 1
            #cu = c.cursor()
            gifts.loc[(gifts['Latitude']>(i-180/n))&(gifts['Latitude']<i)&(gifts['Longitude']>(j-360/n))&(gifts['Longitude']<(j)),"i"]=i_
            gifts.loc[(gifts['Latitude']>(i-180/n))&(gifts['Latitude']<i)&(gifts['Longitude']>(j-360/n))&(gifts['Longitude']<(j)),"j"]=j_

    
    for limit_ in [70]:

        trips=gifts[gifts['TripId']==0]
        trips=trips.sort_values(['i','j','Longitude','Latitude'])
        trips=trips[0:limit_]
        t_ = 0
        while len(trips.GiftId)>0:
            g = []
            t_ += 1
            w_ = 0.0
            for i in range(len(trips.GiftId)):
                    if (w_ + float(trips.iloc[i,3]))<= weight_limit:
                        w_ += float(trips.iloc[i,3])
                        g.append(trips.iloc[i,0])
            gifts.loc[gifts['GiftId'].isin(g),'TripId']=t_
            trips=gifts[gifts['TripId']==0]
            trips=trips.sort_values(['i','j','Longitude','Latitude'])
            trips=trips[0:limit_]

        
        ou_ = open("submission_opt" + str(limit_) + " " + str(n) + ".csv","w")
        ou_.write("TripId,GiftId\n")
        bm = 0.0

        for s_ in range(1,t_+1):
            trip=gifts[gifts['TripId']==s_]
            trip=trip.sort_values(['Latitude','Longitude'],ascending=[0,1])
            
            a = []
            for x_ in range(len(trip.GiftId)):
               
                a.append([trip.iloc[x_,0],(trip.iloc[x_,1],trip.iloc[x_,2]),trip.iloc[x_,3]])
            b = bb_sort(a)
            if path_opt_test(a) <= path_opt_test(b):
                print("TripId",s_, "No Change", path_opt_test(a) , path_opt_test(b))
                bm += path_opt_test(a)
                for y_ in range(len(a)):
                    ou_.write(str(s_)+","+str(a[y_][0])+"\n")
            else:
                print("TripId ", s_, "Optimized", path_opt_test(a) - path_opt_test(b))
                bm += path_opt_test(b)
                for y_ in range(len(b)):
                    ou_.write(str(s_)+","+str(b[y_][0])+"\n")
        ou_.close()
        
        benchmark = 12506999134.2
        if bm < benchmark:
            print(n, limit_, "Improvement", bm, bm - benchmark, benchmark)
        else:
            print(n, limit_, "Try again", bm, bm - benchmark, benchmark)

TripId 1 No Change 5367299.72567 5367299.72567
TripId 2 No Change 5551026.65884 5551026.65884
TripId 3 No Change 4914543.75948 4914543.75948
TripId 4 No Change 5282952.53287 5282952.53287
TripId  5 Optimized 58.8093977934
TripId 6 No Change 5159080.59761 5159080.59761
TripId 7 No Change 5081695.99851 5081695.99851
TripId 8 No Change 5352986.77545 5352986.77545
TripId 9 No Change 5038477.31756 5038477.31756
TripId  10 Optimized 952.624528087
TripId 11 No Change 5953722.02837 5953722.02837
TripId 12 No Change 5953881.21837 5953881.21837
TripId 13 No Change 5733764.86121 5733764.86121
TripId 14 No Change 5049921.97243 5049921.97243
TripId  15 Optimized 836.178852434
TripId  16 Optimized 173.771359297
TripId 17 No Change 4905473.01407 4905473.01407
TripId 18 No Change 4957984.90498 4957984.90498
TripId 19 No Change 4969690.29834 4969690.29834
TripId  20 Optimized 224.327446859
TripId 21 No Change 3737602.30594 3737602.30594
TripId 22 No Change 5306477.00136 5306477.00136
TripId 23 No Chang

In [3]:
print("submission_opt" + str(limit_) + " " + str(n) + ".csv")

submission_opt70 1.2523.csv
